<h1 style="color:rgb(0,120,170)">Artificial Intelligence in Life Sciences</h1>
<h2 style="color:rgb(0,120,170)">Evaluation and assessment of generative models</h2>

#Importing Libraries

In [1]:
!pip install rdkit
!pip install fire 
!pip install fcd

# Neural-based generator of organic compounds
import functools
import fire
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.functional import one_hot
from tqdm import tqdm
from rdkit import RDLogger
from rdkit import Chem


import os
import pandas as pd
import numpy as np
import pickle
import rdkit

# Ignore some warnings from RDKIT and keras
from rdkit import RDLogger  
RDLogger.DisableLog('rdApp.*') 

import warnings
warnings.filterwarnings("ignore")

# Load methods from the FCD library
from fcd import get_fcd, load_ref_model, canonical_smiles, get_predictions, calculate_frechet_distance

np.random.seed(1234)

print("RDKit: ", rdkit.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 38.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116932 sha256=c2d145b93e2d47f90a69e2052c78b503a53d7076e7c8bc04ee0b41599dc05672
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 MB 10.5 MB/s eta 0:00:00
RDKit:  2023.03.1


# GRU model

In [ ]:
__special__ = {0: "<PAD>", 1: "<BOS>", 2: "<EOS>"}
RDLogger.DisableLog('rdApp.*')

class SmilesProvider(torch.utils.data.DataLoader):
    def __init__(self,file,total=130):
        self.total = total
        self.smiles = open(file, 'r').read().split("\n")[:-1]
        tokens = functools.reduce(lambda acc,s: acc.union(set(s)), self.smiles ,set())
        self.vocsize = len(tokens) + len(__special__)
        self.index2token = dict(enumerate(tokens,start=3))
        self.index2token.update(__special__)
        self.token2index = {v:k for k,v in self.index2token.items()}
        self.ints = [torch.LongTensor([self.token2index[s] for s in line]) for line in tqdm(self.smiles,"Preparing of a dataset")]

    def decode(self,indexes):
        return "".join([self.index2token[index] for index in indexes if index not in __special__])

    def __getitem__(self,i):
        special_added = torch.cat((torch.LongTensor([self.token2index['<BOS>']])
                                   ,self.ints[i],torch.LongTensor([self.token2index['<EOS>']]),
                                   torch.LongTensor([self.token2index["<PAD>"]]*(self.total-len(self.ints[i])-2))),dim=0)
        return one_hot(special_added,self.vocsize).float(),special_added

    def __len__(self):
        return len(self.smiles)

class SimpleGRU(nn.Module):

    def __init__(self, vocsize,device,hidden_size=512,num_layers=3):
        super().__init__()
        self.device = device
        self.vocsize = vocsize
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(vocsize,hidden_size,bidirectional=False,batch_first=True,num_layers=num_layers)
        self.linear = nn.Linear(hidden_size,vocsize)


    def forward(self,x):
        output = self.gru(x)[0]
        final = self.linear(output)
        return final

    def sample(self,batch_size=128,max_len=130):
        bos_token = [k for k,v in __special__.items() if v == "<BOS>"][0]
        x = torch.LongTensor([bos_token]*batch_size)
        h = torch.zeros((self.num_layers,batch_size,self.hidden_size)).to(self.device)
        accumulator = torch.zeros(batch_size,max_len)
        for i in range(max_len):
            x = one_hot(x, self.vocsize).float().unsqueeze(1).to(self.device)
            output,h = self.gru(x,h)
            next = F.softmax(self.linear(output).squeeze(1),dim=1)
            x = torch.multinomial(next,num_samples=1,replacement=True).squeeze(1)
            accumulator[:,i] = x
        return accumulator

def generate(file='genmodel_.pt', batch_size=64):
    box = torch.load(file)
    model, tokenizer = box['model'], box['tokenizer']
    model.eval()
    res = model.sample(batch_size)
    correct = 0
    for i in range(res.size(0)):
        smiles = "".join([tokenizer[index] for index in res[i].tolist() if index not in __special__])
        print(smiles)
        correct = correct + 1 if Chem.MolFromSmiles(smiles) else correct

    print("% of correct molecules is {:4.2f}".format(correct / float(batch_size) * 100))



def train(file='/content/smiles_train-2.txt', batch_size=256, learning_rate=0.001, n_epochs=2, device='cuda'):
    device = device if torch.cuda.is_available() else 'cpu'
    dataset = SmilesProvider(file)
    model = SimpleGRU(dataset.vocsize, device=device).to(device)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    loss_function = nn.CrossEntropyLoss()
    model.train()
    for epoch in range(1, n_epochs + 1):
        for iteration, (batch, target) in enumerate(tqdm(dataloader, 'Training')):
            batch, target = batch.to(device), target.to(device)
            out = model(batch)
            out = out.transpose(2, 1)
            loss = loss_function(out[:, :, :-1], target[:, 1:])
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    model.device = 'cpu'
    torch.save({'tokenizer': dataset.index2token, 'model': model.cpu()}, "genmodel.pt")


#if __name__ == '__main__':
    #fire.Fire()

#Train model

In [ ]:
if __name__ == '__main__':
    train()

#Generate some new molecules 

In [ ]:
for _ in range(25000):
    generate(file='genmodel.pt', batch_size=1)

CC(=O)c1ccccc1CN1C(=O)c2ccc(C(=O)Nc3cccc(C(=O)O)c3)cc2CC1C
% of correct molecules is 100.00
c1cc(-c2nnc(-c3ccccc3Nc3nncs3)n2-c2ccccc2)cc1
% of correct molecules is 0.00
Cc1cccc(C(=O)C=Cc2sccc2-c2nnn[nH]2)c1
% of correct molecules is 100.00
COc1ccc(NC(=O)c2cccc(OC(=O)c3cccc(OC)c3)c2)cc1
% of correct molecules is 100.00
S=CNSc1nc2ccccc2o1
% of correct molecules is 100.00
Cn1c(=O)c2ccc3c(c21)C(=O)c1ccccc1-3
% of correct molecules is 100.00
CCCC(CN1CC=CCC1)NS(=O)(=O)c1ccc(-c2ccccc2)cc1
% of correct molecules is 100.00
Cc1cccc(NNC(=O)C(N)Cc2ccc(OCCCCNc3ccc(Cl)cc3C)o2)c1
% of correct molecules is 100.00
CC(=O)CCn1nc(C)cc1C(=O)NCc1cccs1
% of correct molecules is 100.00
CCc1ccc(CNC(=O)CC(=O)N=C(N)N)cc1
% of correct molecules is 100.00
Oc1cc2c(cc1O)Oc1cccc(OCCNc3c5c(nc5ccccc25)CCC35)c1
% of correct molecules is 100.00
Cc1sc2ncn(Cc3ccccc3)c2c1CN1CCN(c2ccccc2)CC1
% of correct molecules is 100.00
CCc1cc2ncc(C=Cc3ccc(O)cc3)sc2c1C
% of correct molecules is 100.00
CC12CCC3C(CCCC3)CC1C(NC(=O)c1cccs1)C

#Valid molecules filtering 

In [ ]:
import re

input_file = '/content/samples_file.txt'
output_file = 'correct of 20k samples.txt'
target_phrase = '% of correct molecules is 100.00'

# Open the input file for reading
with open(input_file, 'r') as file:
    lines = file.readlines()

# Extract the molecules above the target phrase
correct_molecules = []
for i in range(1, len(lines)):
    line = lines[i].strip()
    if line.startswith(target_phrase):
        molecule = lines[i - 1].strip()
        correct_molecules.append(molecule)

# Create a new file with the correct molecules
with open(output_file, 'w') as file:
    file.write('\n'.join(correct_molecules))


<h2 style="color:rgb(0,120,170)">Fréchet ChemNet Distance (FCD)</h2>

In [4]:
# Load ChemNet model
model = load_ref_model()

Now we can load some generated SMILES strings and calculate the FCD between two smaller subsets sampled from this set. The FCD calculates the distance from a set of `real` samples to a set of `generated` samples. In this first step we just assume that the first set we sample are the `real` samples and the second one the `generated` samples.

In [2]:
# Load generated molecules from an input file, which contains one generated SMILES per line
gen_mol_file = "/content/correct of 20k samples.txt"
gen_mol = pd.read_csv(gen_mol_file,header=None)[0] 

# Sample two subsets, treat one as "real" SMILES and the other as "generated" SMILES for now
smiles_real = np.random.choice(gen_mol, 8500, replace=False)
smiles_gen = np.random.choice(gen_mol, 8500, replace=False)

# get canonical smiles and filter invalid ones
smiles_real_can = [w for w in canonical_smiles(smiles_real) if w is not None]
smiles_gen_can = [w for w in canonical_smiles(smiles_gen) if w is not None]

In [6]:
fcd_value = get_fcd(smiles_real_can, smiles_gen_can, model=model)
print('FCD: ', fcd_value)

FCD:  0.17342240055781133


To show that canonicalization is important, let's see what happens if we don't canonicalize our generated molecules.

In [7]:
fcd_value = get_fcd(smiles_real_can, smiles_gen, model=model)
print('FCD: ', fcd_value)

FCD:  0.19146534035552065


<h1 style="color:rgb(0,120,170)">Challenge Server Evaluation</h1>

For the molecule generation exercise you need to submit a set of at least 10000 SMILES strings generated by a model you trained.

In [8]:
# Load sample submission
gen_mol_file = "/content/correct of 20k samples.txt"
smiles_gen = pd.read_csv(gen_mol_file, header=None)[0].iloc[:10000] 

# get canonical smiles and filter invalid ones
smiles_gen_can = [w for w in canonical_smiles(smiles_gen) if w is not None]

In [9]:
# Calculate statistics for the sample submission
act_gen = get_predictions(model, smiles_gen_can)
mu_gen = np.mean(act_gen, axis=0)
sigma_gen = np.cov(act_gen.T)

# Load precomputed test mean and covariance
with open("/content/test_stats.p", 'rb') as f:
    mu_test, sigma_test = pickle.load(f)

Now we can calculate the FCD between the sample submission and real molecules.

In [10]:
fcd_value = calculate_frechet_distance(
        mu1=mu_gen,
        mu2=mu_test,
        sigma1=sigma_gen,
        sigma2=sigma_test)
print('FCD: ', fcd_value)

FCD:  1.9756214127676799


<h2 style="color:rgb(0,120,170)">Validity</h2>

Another metric we can calculate for a submission is the ratio of valid SMILES strings.

In [11]:
validity = len(smiles_gen_can) / len(smiles_gen)
print("Validity: ", validity)

Validity:  1.0


<h2 style="color:rgb(0,120,170)">Uniqueness</h2>

We can also calculate the ratio of unique molecules generated by the model. If the model "collapses" and just generates the same valid molecule over and over we would see that in this metric.

In [12]:
smiles_unique = set(smiles_gen_can)
uniqueness = len(smiles_unique) / len(smiles_gen)
print("Uniqueness: ", uniqueness)

Uniqueness:  0.999


<h2 style="color:rgb(0,120,170)">Novelty</h2>

Lastly, we can check if the generated molecules are indeed "new", or novel, or if the model copied molecules from the training set by comparing the generated molecules to the molecules in the training set.

In [14]:
# load training set for novelty
with open("/content/smiles_train-2.txt") as f:
    smiles_train = {s for s in f.read().split() if s}
    
smiles_novel = smiles_unique - smiles_train
novelty = len(smiles_novel) / len(smiles_gen)
print("Novelty: ", novelty)

Novelty:  0.9839
